In [ ]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
import lime
import lime.lime_tabular
import numpy as np

In [ ]:
columns = ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
           'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target']

data = pd.read_csv("processed.cleveland.data", names=columns)

# Drop rows where "?" is present as a value
data = data.drop(data[data.slope == "?"].index)
data = data.drop(data[data.ca == "?"].index)
data = data.drop(data[data.thal == "?"].index)

convert = ['slope','ca','thal']
data[convert] = data[convert].astype(float) # data types for each column must be numeric

In [ ]:
x = data.drop('target', axis=1)
y = data['target']

# Scale the data so everything is between 0 and 1
x_scaled = x / x.max() # maximum is taken per column

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_scaled, y, test_size=0.1, random_state=11)

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train.values, y_train.values))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test.values, y_test.values))

In [ ]:
train_dataset = train_dataset.shuffle(buffer_size=len(x_train)).batch(32)
test_dataset = test_dataset.batch(32)

TensorFlow: https://www.tensorflow.org/api_docs/python/tf/keras/Sequential

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(14, activation='sigmoid', input_shape=(x_train.shape[1],)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(5, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.01),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

# Train the model
model.fit(train_dataset, epochs=100)

In [ ]:
preds = model.evaluate(train_dataset)

LIME: https://github.com/marcotcr/lime

In [ ]:
explainer = lime.lime_tabular.LimeTabularExplainer(x_train.to_numpy(),
                                                   feature_names=columns,
                                                   class_names=["0","1","2","3","4"],
                                                   discretize_continuous=True)

In [ ]:
sample_num = 11
exp = explainer.explain_instance(x_test.to_numpy()[sample_num], model.predict, num_features=5, top_labels=1)

In [ ]:
exp.show_in_notebook(show_table=True, show_all=False)